In [1]:
# First, load you model and allocate the tensors
import tensorflow.lite as tflite
# import tflite_runtime.interpreter as tflite
# Load TFLite model and allocate tensors.
interpreter = tflite.Interpreter(model_path='../data/model_dir/model.tflite')

In [2]:
#allocate the tensors
interpreter.allocate_tensors()

# Get the input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [3]:
input_details

[{'name': 'input_1',
  'index': 0,
  'shape': array([  1, 224, 224,   3]),
  'shape_signature': array([ -1, 224, 224,   3]),
  'dtype': numpy.uint8,
  'quantization': (0.003921568859368563, 0),
  'quantization_parameters': {'scales': array([0.00392157], dtype=float32),
   'zero_points': array([0]),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [4]:
output_details

[{'name': 'Identity',
  'index': 168,
  'shape': array([1, 5]),
  'shape_signature': array([-1,  5]),
  'dtype': numpy.uint8,
  'quantization': (0.00390625, 0),
  'quantization_parameters': {'scales': array([0.00390625], dtype=float32),
   'zero_points': array([0]),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [5]:
image_path = os.path.join("d:/","oneNeuron/Source/tflite_testing/daisy.jpeg")

In [6]:
import os
os.path.isfile(image_path)

True

In [7]:
import cv2
import numpy as np
import imghdr

In [8]:

if imghdr.what(image_path) not in ["jpeg", "jpg", "png"]:
    print("quiting the program")
img = cv2.imread(image_path)
img = cv2.resize(img,(224,224))

In [9]:
img.shape

(224, 224, 3)

In [10]:
input_shape = input_details[0]['shape']
input_shape

array([  1, 224, 224,   3])

In [11]:
input_tensor= np.array(np.expand_dims(img,0))
input_tensor

array([[[[  5, 117,  83],
         [  6, 116,  82],
         [  7, 114,  81],
         ...,
         [ 12,  83,  10],
         [  7,  78,   5],
         [  2,  74,   0]],

        [[  5, 117,  83],
         [  5, 116,  82],
         [  6, 113,  80],
         ...,
         [ 11,  82,   9],
         [  7,  78,   5],
         [  3,  74,   1]],

        [[  5, 117,  82],
         [  4, 115,  81],
         [  6, 113,  80],
         ...,
         [ 10,  82,   9],
         [  6,  78,   5],
         [  3,  74,   1]],

        ...,

        [[  6,  95,  26],
         [  5,  95,  28],
         [  5,  95,  29],
         ...,
         [  3,  50,   3],
         [  3,  50,   3],
         [  3,  49,   3]],

        [[  6,  94,  25],
         [  5,  95,  27],
         [  4,  94,  28],
         ...,
         [  4,  50,   4],
         [  3,  50,   3],
         [  3,  49,   3]],

        [[  5,  94,  24],
         [  5,  94,  27],
         [  4,  94,  28],
         ...,
         [  4,  50,   4],
        

In [12]:
# Set the tensor to point to the input data to be inferred, then run the inference
input_index = interpreter.get_input_details()[0]["index"]
input_index

0

In [13]:
interpreter.set_tensor(input_index, input_tensor)
interpreter.invoke()
output_details = interpreter.get_output_details()
output_details

[{'name': 'Identity',
  'index': 168,
  'shape': array([1, 5]),
  'shape_signature': array([-1,  5]),
  'dtype': numpy.uint8,
  'quantization': (0.00390625, 0),
  'quantization_parameters': {'scales': array([0.00390625], dtype=float32),
   'zero_points': array([0]),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [14]:
# Make the prediction
output_data = interpreter.get_tensor(output_details[0]['index'])
pred = np.squeeze(output_data)
pred

array([237,   3,   2,   5,   8], dtype=uint8)

In [15]:
# If you print “pred”, you will get an array of uint8 (0–255) values. These correspond to the confidence for each class. In my example that would be the 16 bird species we specified in the Collecting Image Data For Machine Learning in Python tutorial.
# To get a more human friendly prediction, specify the class indices below.
class_ind = {
  0: "daisy",
  1: "dandelion",
  2: "roses",
  3: "sunflowers",
  4: "tulips",}
# # Since “pred” corresponds to the confidence of each bird class, our prediction should be the highest value in the array. Grab the highest prediction location.
highest_pred_loc = np.argmax(pred)
print(highest_pred_loc, pred)

0 [237   3   2   5   8]


In [16]:
# # Use the “highest_pred_loc” to search the “class_ind” dictionary for the actual bird’s name. Print out the prediction and you’re done!
flower_name = class_ind[highest_pred_loc]
print(f"predicted flower is {flower_name}")

predicted flower is daisy


In [43]:
import tensorflow.lite as tflite
import os
import cv2
import numpy as np
import imghdr
import sys

def predict(config, label_map):
    # model_path = config["train"]["model_path"]
    img_path = config["predict"]["test_img_path"]
    interpreter = tflite.Interpreter(model_path="../data/model_dir/model.tflite")
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    img_type = imghdr.what(img_path)
    a_file = os.path.isfile(img_path)
    if (img_type not in ["jpeg", "png", "jpg"]) or (not a_file):
        print("quiting the program")
        sys.exit()
    else:
        print(f"yes the file is of type: {img_type}. proceeding further")
    batch_size, pixel_ht, pexel_wd, ch = input_details[0]['shape']
    img = cv2.imread(img_path)
    img = cv2.resize(img, (pixel_ht, pexel_wd))
    input_img = np.expand_dims(img, 0)
    input_index = input_details[0]['index']
    interpreter.set_tensor(input_index, input_img)
    interpreter.invoke()
    output_index = output_details[0]['index']
    output = interpreter.get_tensor(output_index)
    pred =  np.squeeze(output)
    argmax = np.argmax(pred)
    result = label_map[argmax]
    return result, argmax

In [44]:
from src.utils.common import read_yaml

config = read_yaml("../config/config.yaml")

In [45]:
label_map = {
  0: "daisy",
  1: "dandelion",
  2: "roses",
  3: "sunflowers",
  4: "tulips",}

In [46]:
predict(config, label_map)

yes the file is of type: jpeg. proceeding further


('daisy', 0)

In [27]:
config


{'train': {'data_dir': 'data',
  'data_split': 0.9,
  'model_file_dir': 'data/model_dir',
  'data_file_name': 'flower_photos.tgz',
  'base_url': 'https://storage.googleapis.com/download.tensorflow.org/example_images/'},
 'predict': {'test_img': 'd:/oneNeuron/Source/tflite_testing/daisy.jpeg'}}

In [31]:
from pathlib import Path

In [34]:
read_yaml(Path("../config/config.yaml"))

{'train': {'data_dir': 'data',
  'data_split': 0.9,
  'model_file_dir': 'data/model_dir',
  'data_file_name': 'flower_photos.tgz',
  'base_url': 'https://storage.googleapis.com/download.tensorflow.org/example_images/'},
 'predict': {'test_img': 'd:/oneNeuron/Source/tflite_testing/daisy.jpeg'}}

In [28]:
cv2.imread(config["predict"]["test_img"])

array([[[  5, 117,  83],
        [  5, 117,  83],
        [  6, 116,  82],
        ...,
        [  9,  80,   7],
        [  5,  76,   3],
        [  2,  73,   0]],

       [[  5, 117,  83],
        [  4, 116,  82],
        [  5, 115,  81],
        ...,
        [  9,  80,   7],
        [  5,  76,   3],
        [  3,  74,   1]],

       [[  4, 116,  82],
        [  4, 116,  82],
        [  5, 115,  81],
        ...,
        [  6,  79,   6],
        [  4,  77,   4],
        [  1,  74,   1]],

       ...,

       [[  5,  96,  27],
        [  5,  95,  29],
        [  5,  95,  30],
        ...,
        [  3,  51,   3],
        [  2,  50,   2],
        [  2,  50,   2]],

       [[  6,  95,  25],
        [  6,  95,  25],
        [  5,  95,  29],
        ...,
        [  4,  50,   4],
        [  3,  49,   3],
        [  3,  49,   3]],

       [[  5,  94,  24],
        [  6,  95,  25],
        [  4,  94,  28],
        ...,
        [  4,  50,   4],
        [  3,  49,   3],
        [  3,  49,   3]]